In [ ]:
import geopandas as gpd
import pandas as pd

# Read the GeoJSON file as a GeoDataFrame
gdf = gpd.read_file('cycleway_base.geojson')

# Display basic information
print(f"Total features: {len(gdf)}")
print(f"\nColumns: {gdf.columns.tolist()}")

# Check if we have contrib_type or length_delta fields
if 'contrib_type' in gdf.columns:
    print(f"\nContrib type distribution:")
    print(gdf['contrib_type'].value_counts())
if 'length_delta' in gdf.columns:
    print(f"\nLength delta stats:")
    print(gdf['length_delta'].describe())

print(f"\nFirst few rows:")
gdf.head()


Total features: 2549

Columns: ['wkt', 'tags', 'changeset', 'usr', 'changeset_id', 'highway', 'hashtags', 'user_name', 'user_plot', 'geometry']

First few rows:


,wkt,tags,changeset,usr,changeset_id,highway,hashtags,user_name,user_plot,geometry
0,"LINESTRING (11.8477091 50.8897581, 11.847593 5...","{ ""surface"": ""asphalt"", ""sidewalk:right"": ""sep...","{'id': 171948163, 'created_at': datetime.datet...","{ ""id"": 3644159, ""name"": ""bicyclett"" }",171948163,secondary,#missing-cw_mapillary-signs;#maproulette,bicyclett,bicyclett,"LINESTRING (11.84771 50.88976, 11.84759 50.88993)"
1,"LINESTRING (8.1118937 49.1907098, 8.1118295 49...","{ ""surface"": ""asphalt"", ""maxspeed"": ""30"", ""one...","{'id': 172194455, 'created_at': datetime.datet...","{ ""id"": 3644159, ""name"": ""bicyclett"" }",172194455,residential,#missing-cw_mapillary-signs;#maproulette,bicyclett,bicyclett,"LINESTRING (8.11189 49.19071, 8.11183 49.19095..."
2,"LINESTRING (10.0998905 52.8451411, 10.0995923 ...","{ ""sidewalk"": ""left"", ""lcn"": ""yes"", ""surface"":...","{'id': 170571393, 'created_at': datetime.datet...","{ ""id"": 3644159, ""name"": ""bicyclett"" }",170571393,service,#missing-cw_mapillary-signs;#maproulette,bicyclett,bicyclett,"LINESTRING (10.09989 52.84514, 10.09959 52.845..."
3,"LINESTRING (9.209622 48.0880089, 9.2093329 48....","{ ""traffic_sign"": ""DE:240"", ""bicycle"": ""design...","{'id': 172234852, 'created_at': datetime.datet...","{ ""id"": 3644159, ""name"": ""bicyclett"" }",172234852,path,#missing-cw_mapillary-signs;#maproulette,bicyclett,bicyclett,"LINESTRING (9.20962 48.08801, 9.20933 48.08823..."
4,"LINESTRING (6.8664715 51.6906954, 6.8661547 51...","{ ""foot"": ""designated"", ""surface"": ""paving_sto...","{'id': 172201215, 'created_at': datetime.datet...","{ ""id"": 3644159, ""name"": ""bicyclett"" }",172201215,path,#missing-cw_mapillary-signs;#maproulette,bicyclett,bicyclett,"LINESTRING (6.86647 51.6907, 6.86615 51.69058,..."


In [ ]:
print  ("test")

test


In [ ]:
import json
import ast

# Parse the tags column to extract key-value pairs
def parse_tags(tags_str):
    """Parse the tags JSON string into a dictionary"""
    try:
        # Try to parse as JSON
        tags = json.loads(tags_str)
    except:
        try:
            # Try to parse as Python dict string
            tags = ast.literal_eval(tags_str)
        except:
            return {}
    return tags

# Parse tags for each row
gdf['tags_parsed'] = gdf['tags'].apply(parse_tags)

# Function to categorize features based on highway and cycleway tags
def categorize_feature(tags_dict):
    """Categorize a feature based on its tags"""
    highway = tags_dict.get('highway', '')
    bicycle = tags_dict.get('bicycle', '')
    cycleway = tags_dict.get('cycleway', '')
    cycleway_left = tags_dict.get('cycleway:left', '')
    cycleway_right = tags_dict.get('cycleway:right', '')
    bicycle_type = tags_dict.get('bicycle_road', '')
    
    # Check for cycleway tags first
    if cycleway in ['track', 'lane', 'share_busway', 'advisory']:
        return f"cw={cycleway}"
    elif cycleway_left in ['track', 'lane', 'share_busway', 'advisory']:
        return f"cw_left={cycleway_left}"
    elif cycleway_right in ['track', 'lane', 'share_busway', 'advisory']:
        return f"cw_right={cycleway_right}"
    
    # Check for bicycle road
    if bicycle_type or (highway in ['residential', 'tertiary', 'secondary'] and bicycle == 'designated'):
        return 'bicycle_road'
    
    # Check for track with bicycle
    if highway == 'track' and bicycle == 'designated':
        return 'hw=track_bicycle=designated'
    
    # Check for footway with bicycle
    if highway == 'footway' and bicycle == 'yes':
        return 'hw=footway_bicycle=yes'
    
    # Check for path with bicycle
    if highway == 'path' and bicycle == 'designated':
        return 'hw=path_bicycle=designated'
    
    # Check for cycleway highway
    if highway == 'cycleway':
        return 'hw=cycleway'
    
    return 'other'

# Add category column
gdf['category'] = gdf['tags_parsed'].apply(categorize_feature)

# Display category distribution
print(f"CRS: {gdf.crs}")
print(f"\nGeometry types: {gdf.geom_type.value_counts()}")
print(f"\nCategory distribution:")
print(gdf['category'].value_counts())


CRS: EPSG:4326

Geometry types: LineString    2549
Name: count, dtype: int64

Category distribution:
category
other                          1348
hw=path_bicycle=designated      684
cw_right=track                  208
hw=footway_bicycle=yes          102
cw_right=lane                    87
hw=cycleway                      48
cw_left=track                    28
bicycle_road                     18
hw=track_bicycle=designated      12
cw_left=lane                     12
cw=lane                           2
Name: count, dtype: int64


In [ ]:
# Parse changeset to determine if feature was added or tag-adjusted
def parse_changeset(changeset_str):
    """Parse changeset string to get basic info"""
    try:
        changeset_dict = ast.literal_eval(changeset_str)
        return {
            'id': changeset_dict.get('id', None),
            'tags': changeset_dict.get('tags', {}),
            'numChanges': changeset_dict.get('numChanges', 0)
        }
    except:
        return {'id': None, 'tags': {}, 'numChanges': 0}

# Apply changeset parsing
changeset_info = gdf['changeset'].apply(parse_changeset)
gdf['changeset_tags'] = changeset_info.apply(lambda x: x.get('tags', {}))
gdf['changeset_num_changes'] = changeset_info.apply(lambda x: x.get('numChanges', 0))

# Extract comment from changeset tags
gdf['changeset_comment'] = gdf['changeset_tags'].apply(
    lambda x: x.get('comment', '') if isinstance(x, dict) else ''
)

# Show sample of comments to understand the data
print("Sample changeset comments:")
print(gdf['changeset_comment'].head(10))
print(f"\nComment value counts (first 5):")
print(gdf['changeset_comment'].value_counts().head())

# Determine if feature was likely added (new geometry) or tag-adjusted
def determine_action(comment, num_changes):
    """Determine if feature was added or tag-adjusted"""
    comment_lower = str(comment).lower()
    
    # Keywords that suggest new geometry was created
    if any(keyword in comment_lower for keyword in ['added', 'add', 'create', 'created', 'new']):
        return 'added'
    # Keywords that suggest only tags were modified
    elif any(keyword in comment_lower for keyword in ['tag', 'tags', 'updated tag', 'adjust tag']):
        return 'tag_adjusted'
    else:
        # Default heuristic: based on your original table, most should be "added"
        # If there are multiple changes, it's likely a new feature being created
        if num_changes > 3:
            return 'added'
        else:
            return 'tag_adjusted'

gdf['action_type'] = gdf.apply(lambda row: determine_action(
    row['changeset_comment'], row['changeset_num_changes']
), axis=1)

print(f"\nAction type distribution:")
print(gdf['action_type'].value_counts())

# Filter to the categories you're interested in
target_categories = [
    'hw=path_bicycle=designated',
    'hw=cycleway',
    'hw=footway_bicycle=yes',
    'hw=track_bicycle=designated',
    'bicycle_road',
    'cw=track',
    'cw=lane'
]

# Get filtered GeoDataFrame
gdf_filtered = gdf[gdf['category'].isin(target_categories)].copy()

# Display results
print(f"\nTotal features in gdf: {len(gdf)}")
print(f"Features matching target categories: {len(gdf_filtered)}")
print(f"\nCategories found:")
print(gdf_filtered['category'].value_counts())
print(f"\nAction types:")
print(gdf_filtered['action_type'].value_counts())

# Show sample
print("\nSample of filtered data:")
gdf_filtered[['category', 'highway', 'action_type', 'changeset_id']].head(10)



Total features in gdf: 2549
Features matching target categories: 866

Categories found:
category
hw=path_bicycle=designated     684
hw=footway_bicycle=yes         102
hw=cycleway                     48
bicycle_road                    18
hw=track_bicycle=designated     12
cw=lane                          2
Name: count, dtype: int64

Action types:
action_type
tag_adjusted    866
Name: count, dtype: int64

Sample of filtered data:


,category,highway,action_type,changeset_id
3,hw=path_bicycle=designated,path,tag_adjusted,172234852
4,hw=path_bicycle=designated,path,tag_adjusted,172201215
5,hw=path_bicycle=designated,path,tag_adjusted,170864934
8,hw=path_bicycle=designated,path,tag_adjusted,171869466
9,hw=path_bicycle=designated,path,tag_adjusted,171539681
10,hw=path_bicycle=designated,path,tag_adjusted,170557249
14,hw=path_bicycle=designated,path,tag_adjusted,172306269
17,hw=path_bicycle=designated,path,tag_adjusted,171948163
26,bicycle_road,secondary,tag_adjusted,171644343
27,hw=path_bicycle=designated,path,tag_adjusted,170141146


In [5]:
# Calculate length in km for each feature
# Assuming geometry is in WGS84 (CRS84), convert to a projected CRS for accurate distance calculation
gdf_filtered_projected = gdf_filtered.to_crs('EPSG:3857')  # Web Mercator
gdf_filtered_projected['length_km'] = gdf_filtered_projected.geometry.length / 1000

# Create summary by category and action type
summary = gdf_filtered_projected.groupby(['category', 'action_type'])['length_km'].sum().unstack(fill_value=0)

# Ensure all categories are present
for cat in target_categories:
    if cat not in summary.index:
        summary.loc[cat] = 0

# Reorder to match your table
summary = summary.reindex(target_categories)

print("Summary by category and action type (km):")
print(summary.round(1))

# Display the full filtered geodataframe
print(f"\n\nFull filtered GeoDataFrame ({len(gdf_filtered)} features):")
print(gdf_filtered.columns.tolist())
gdf_filtered.head()


Summary by category and action type (km):
action_type                  tag_adjusted
category                                 
hw=path_bicycle=designated          308.0
hw=cycleway                           8.8
hw=footway_bicycle=yes               24.5
hw=track_bicycle=designated           8.4
bicycle_road                          5.2
cw=track                              0.0
cw=lane                               0.4


Full filtered GeoDataFrame (866 features):
['wkt', 'tags', 'changeset', 'usr', 'changeset_id', 'highway', 'hashtags', 'user_name', 'user_plot', 'geometry', 'tags_parsed', 'category', 'changeset_tags', 'changeset_num_changes', 'action_type']


,wkt,tags,changeset,usr,changeset_id,highway,hashtags,user_name,user_plot,geometry,tags_parsed,category,changeset_tags,changeset_num_changes,action_type
3,"LINESTRING (9.209622 48.0880089, 9.2093329 48....","{ ""traffic_sign"": ""DE:240"", ""bicycle"": ""design...","{'id': 172234852, 'created_at': datetime.datet...","{ ""id"": 3644159, ""name"": ""bicyclett"" }",172234852,path,#missing-cw_mapillary-signs;#maproulette,bicyclett,bicyclett,"LINESTRING (9.20962 48.08801, 9.20933 48.08823...","{'traffic_sign': 'DE:240', 'bicycle': 'designa...",hw=path_bicycle=designated,{},0,tag_adjusted
4,"LINESTRING (6.8664715 51.6906954, 6.8661547 51...","{ ""foot"": ""designated"", ""surface"": ""paving_sto...","{'id': 172201215, 'created_at': datetime.datet...","{ ""id"": 3644159, ""name"": ""bicyclett"" }",172201215,path,#missing-cw_mapillary-signs;#maproulette,bicyclett,bicyclett,"LINESTRING (6.86647 51.6907, 6.86615 51.69058,...","{'foot': 'designated', 'surface': 'paving_ston...",hw=path_bicycle=designated,{},0,tag_adjusted
5,"LINESTRING (12.1848052 50.6285585, 12.1849054 ...","{ ""surface"": ""asphalt"", ""traffic_sign"": ""DE:24...","{'id': 170864934, 'created_at': datetime.datet...","{ ""id"": 3644159, ""name"": ""bicyclett"" }",170864934,path,#missing-cw_mapillary-signs;#maproulette,bicyclett,bicyclett,"LINESTRING (12.18481 50.62856, 12.18491 50.628...","{'surface': 'asphalt', 'traffic_sign': 'DE:240...",hw=path_bicycle=designated,{},0,tag_adjusted
8,"LINESTRING (6.0614718 50.9534345, 6.0615922 50...","{ ""traffic_sign"": ""DE:240"", ""bicycle"": ""design...","{'id': 171869466, 'created_at': datetime.datet...","{ ""id"": 3644159, ""name"": ""bicyclett"" }",171869466,path,#missing-cw_mapillary-signs;#maproulette,bicyclett,bicyclett,"LINESTRING (6.06147 50.95343, 6.06159 50.95345...","{'traffic_sign': 'DE:240', 'bicycle': 'designa...",hw=path_bicycle=designated,{},0,tag_adjusted
9,"LINESTRING (12.4154308 51.1536795, 12.4154927 ...","{ ""oneway"": ""no"", ""highway"": ""path"", ""bicycle""...","{'id': 171539681, 'created_at': datetime.datet...","{ ""id"": 3644159, ""name"": ""bicyclett"" }",171539681,path,#missing-cw_mapillary-signs;#maproulette,bicyclett,bicyclett,"LINESTRING (12.41543 51.15368, 12.41549 51.153...","{'oneway': 'no', 'highway': 'path', 'bicycle':...",hw=path_bicycle=designated,{},0,tag_adjusted


In [ ]:
# Add length back to the filtered GeoDataFrame in original CRS
gdf_filtered['length_km'] = gdf_filtered_projected['length_km'].values

# Create final summary matching your format
def create_summary(df):
    """Create summary table matching user's format"""
    summary_data = []
    
    # Calculate km by category and action type
    for category in target_categories:
        cat_data = df[df['category'] == category]
        if len(cat_data) > 0:
            km_added = cat_data[cat_data['action_type'] == 'added']['length_km'].sum()
            km_deleted = cat_data[cat_data['action_type'] == 'tag_adjusted']['length_km'].sum()
            summary_data.append({
                'category': category,
                'km_added': round(km_added, 1),
                'km_deleted': round(km_deleted, 1)
            })
        else:
            summary_data.append({
                'category': category,
                'km_added': 0.0,
                'km_deleted': 0.0
            })
    
    return pd.DataFrame(summary_data)

final_summary = create_summary(gdf_filtered_projected)
print("Final summary matching your table format:")
print(final_summary)

# Export summary
final_summary.to_csv('cycleway_summary_by_category.csv', index=False)


Final summary matching your table format:
                      category  km_added  km_deleted
0   hw=path_bicycle=designated       0.0       308.0
1                  hw=cycleway       0.0         8.8
2       hw=footway_bicycle=yes       0.0        24.5
3  hw=track_bicycle=designated       0.0         8.4
4                 bicycle_road       0.0         5.2
5                     cw=track       0.0         0.0
6                      cw=lane       0.0         0.4


In [ ]:
# Save the filtered GeoDataFrame for later use
gdf_filtered.to_file('cycleway_filtered.geojson', driver='GeoJSON')

print(f"\nSaved {len(gdf_filtered)} features to 'cycleway_filtered.geojson'")
print(f"\nGeoDataFrame columns:")
print(list(gdf_filtered.columns))

# Show the category distribution in the filtered dataset
print("\n\nCategory distribution in filtered dataset:")
print(gdf_filtered['category'].value_counts().sort_index())



Saved 866 features to 'cycleway_filtered.geojson'

GeoDataFrame columns:
['wkt', 'tags', 'changeset', 'usr', 'changeset_id', 'highway', 'hashtags', 'user_name', 'user_plot', 'geometry', 'tags_parsed', 'category', 'changeset_tags', 'changeset_num_changes', 'action_type', 'length_km']


Category distribution in filtered dataset:
category
bicycle_road                    18
cw=lane                          2
hw=cycleway                     48
hw=footway_bicycle=yes         102
hw=path_bicycle=designated     684
hw=track_bicycle=designated     12
Name: count, dtype: int64


In [ ]:
# Let's look at what the actual comments say
print("Examining changeset comments:")
sample_comments = gdf['changeset_comment'].value_counts().head(20)
for comment, count in sample_comments.items():
    if len(str(comment)) > 0:
        print(f"{count}: {comment[:80]}")

# The hashtag #missing-cw_mapillary-signs suggests these were ADDED to fix missing cycleways
# So most/all should be classified as "added", not "tag_adjusted"
print("\n\nThe issue: all features have the same hashtag which suggests they were ADDED,")
print("but our logic is classifying them wrong.")


In [ ]:
# The issue: The GeoJSON doesn't have contrib_type or length_delta
# These fields would tell us if a feature was "created" or "modified"
# Without them, we can't properly distinguish "added" from "deleted"

print("Current columns in gdf:")
print(gdf.columns.tolist())

print("\n❌ Missing fields needed to classify added/deleted:")
print("  - contrib_type (would show 'create' vs 'modify')")
print("  - length_delta (would show if geometry was added/subtracted)")

print("\n📝 Solution options:")
print("1. Re-export from DuckDB with contrib_type and length_delta included")
print("2. Use the original Parquet files directly")
print("3. Classify based on heuristic (though this won't be accurate)")


In [ ]:
# To fix this, you need to re-export from DuckDB with contrib_type and length_delta
# Here's the updated query:

import duckdb

# Make sure you have the Parquet files
duckdb.sql("INSTALL spatial; LOAD spatial;")

# Updated query to include contrib_type and length_delta
query = """
SELECT
  ST_GeomFromWKB(geometry) AS geom,
  tags,
  tags_before,
  "user" AS usr,
  changeset,
  contrib_type,  -- This tells us 'create' vs 'modify'
  length,         -- Length of the feature
  length_delta,  -- Change in length (+ = added, - = deleted)
  osm_type,
  osm_id,
  osm_version,
  valid_from
FROM read_parquet({PARQUET_PATHS}, union_by_name=true)
WHERE COALESCE(changeset.tags.hashtags,'') ILIKE '%#missing-cw_mapillary-signs%';
"""

print("Use this DuckDB query to export with contrib_type and length_delta:")
print(query)
print("\nThen save to GeoJSON with:")
print("duckdb.sql('COPY (...) TO \"cycleway_base_with_types.geojson\" (FORMAT GDAL, DRIVER GeoJSON)')")
